In [4]:
import time, os, math, numpy as np, pandas as pd, sklearn as sk, matplotlib as plt

In [5]:
# df_arcos = pd.read_csv("data/arcos_all_washpost.tsv", delimiter='\t')
df_arcos = pd.read_csv('data/arcos_all_washpost.tsv', delimiter='\t', nrows=1000000)
df_hpsa = pd.read_excel("data/Hpsa_Find_Export.xlsx", skiprows=[0,1,2])

<ipython-input-5-9fe2612d50e4>:2: DtypeWarning: Columns (3,5,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_arcos = pd.read_csv('data/arcos_all_washpost.tsv', delimiter='\t', nrows=1000000)


In [6]:
df_arcos.head()

,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,...,DRUG_NAME,Measure,MME_Conversion_Factor,Dosage_Strength,TRANSACTION_DATE,Combined_Labeler_Name,Reporter_family,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,MME
0,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,OXYCODONE,TAB,1.5,10.0,2011-02-08,"Par Pharmaceutical, Inc.",McKesson Corporation,0.89650,100.0,1344.75
1,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,7.5,2011-03-07,SpecGx LLC,McKesson Corporation,0.45405,100.0,454.05
2,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,OXYCODONE,TAB,1.5,10.0,2011-03-10,"Par Pharmaceutical, Inc.",McKesson Corporation,3.58600,400.0,5379.00
3,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,5.0,2011-04-05,SpecGx LLC,McKesson Corporation,1.51350,500.0,1513.50
4,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,5.0,2011-04-06,SpecGx LLC,McKesson Corporation,0.30270,100.0,302.70


In [7]:
df_hpsa.head()

,Discipline,HPSA ID,HPSA Name,Designation Type,Primary State Name,County Name,HPSA FTE Short,HPSA Score,PC MCTA Score,Status,Rural Status,Designation Date,Update Date
0,Primary Care,1016018546,LI-Marion County,Low Income Population HPSA,Alabama,"Marion County, AL",1.673,14,20.0,Designated,Rural,06/22/2022,06/22/2022
1,NaN,Component State Name,Component County Name,Component Name,Component Type,Component GEOID,Component Rural Status,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Alabama,Marion,Marion,Single County,01093,Rural,NaN,NaN,NaN,NaN,NaN,NaN
3,Primary Care,1019011119,Perry County,High Needs Geographic HPSA,Alabama,"Perry County, AL",0.87,19,19.0,Designated,Rural,01/15/1979,09/08/2021
4,NaN,Component State Name,Component County Name,Component Name,Component Type,Component GEOID,Component Rural Status,NaN,NaN,NaN,NaN,NaN,NaN
